## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# LabelEncoder
le = LabelEncoder()

# For the categorical column `Method`, apply LabelEncoder
model_data['Method'] = le.fit_transform(model_data['Method'])

# Define features and target
X = model_data.drop(columns=['Angaus'])
y = model_data['Angaus']

In [3]:
# Split data for early stopping process
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.

In [4]:
# Define XGBoost model
model = xgb.XGBClassifier(n_estimators = 1000, 
                          learning_rate = 0.1, 
                          eval_metric="logloss", 
                          early_stopping_rounds = 50, 
                          random_state=808)

In [5]:
# Fit model
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose = False)

# Get and print the best number of trees
print(f"Best number of trees: {model.best_iteration}")

Best number of trees: 29


### Step 3: Tune Learning Rate

The `learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [6]:
# Initialize model with known best number of trees
model_learn = xgb.XGBClassifier(n_estimators = model.best_iteration,
                                eval_metric="logloss",  
                                random_state=808)

# Define hyperparameter distribution for `learning_rate`
param_learn = {
    "learning_rate": uniform(0.01, 0.3)
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
                                    model_learn, 
                                    param_learn, 
                                    n_iter=20,
                                    scoring="accuracy",
                                    cv=3,
                                    random_state=808)

In [7]:
# Fit model
random_search.fit(X_train, y_train)

# Get and print the best learning rate
print(f"Best learning rate: {random_search.best_params_['learning_rate']:.5f}")

Best learning rate: 0.07485


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [8]:
# Initialize model with known best number of trees and learning rate
model_tune = xgb.XGBClassifier(n_estimators = model.best_iteration,
                               learning_rate = random_search.best_params_['learning_rate'],
                               eval_metric="logloss",  
                               random_state=808)

# Define hyperparameter distributions
param_dist_tune = {
    "max_depth": randint(3, 10),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.05)
}

# Set up RandomizedSearchCV
random_search_tune = RandomizedSearchCV(
    model_tune, 
    param_dist_tune, 
    n_iter=20,
    scoring="accuracy",
    cv=3,
    random_state=808)

In [9]:
# Fit model
random_search_tune.fit(X_train, y_train)

# Get and print the best parameter values
print(f"Best maximum depth: {random_search_tune.best_params_['max_depth']}")
print(f"Best minimum child weight: {random_search_tune.best_params_['min_child_weight']}")
print(f"Best gamma: {random_search_tune.best_params_['gamma']:.5f}")

Best maximum depth: 7
Best minimum child weight: 6
Best gamma: 0.05534


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and 1 (remember to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and 1
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [ ]:
# Initialize model with known best hyperparameter values
model_stoch = xgb.XGBClassifier(n_estimators=model.best_iteration,
                               learning_rate=random_search.best_params_['learning_rate'],
                               max_depth=random_search_tune.best_params_['max_depth'],
                               min_child_weight=random_search_tune.best_params_['min_child_weight'],
                               gamma=random_search_tune.best_params_['gamma'],
                               eval_metric="logloss",  
                               random_state=808)

# Define hyperparameter distributions
param_dist_stoch = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5)
}

# Set up RandomizedSearchCV
random_search_stoch = RandomizedSearchCV(
    model_stoch, 
    param_dist_stoch, 
    n_iter=20,
    scoring="accuracy",
    cv=3,
    random_state=808)

In [ ]:
# Fit model
random_search_stoch.fit(X_train, y_train)

# Get and print the best hyperparameter values
print(f"Best subsample: {random_search_stoch.best_params_['subsample']:.5f}")
print(f"Best colsample_bytree: {random_search_stoch.best_params_['colsample_bytree']:.5f}")


Best subsample: 0.75650
Best colsample_bytree: 0.85066


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [12]:
# For the categorical column `Method`, apply LabelEncoder
eval_data['Method'] = le.fit_transform(eval_data['Method'])

# Define features and target
X_eval = eval_data.drop(columns=['Angaus_obs'])
y_eval = eval_data['Angaus_obs']

In [13]:
model_best = xgb.XGBClassifier(n_estimators=model.best_iteration,
                               learning_rate=random_search.best_params_['learning_rate'],
                               max_depth=random_search_tune.best_params_['max_depth'],
                               min_child_weight=random_search_tune.best_params_['min_child_weight'],
                               gamma=random_search_tune.best_params_['gamma'],
                               subsample=random_search_stoch.best_params_['subsample'],
                               colsample_bytree=random_search_stoch.best_params_['colsample_bytree'],
                               eval_metric="logloss",  
                               random_state=808)

# Train model using the best parameters
model_best.fit(X, y)

# Predict using the evaluation data
model_best.predict(X_eval)

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [14]:
# Predict probability for the positive class
y_pred_proba = model_best.predict_proba(X_eval)[:, 1]  

# Compute the AUC 
auc = roc_auc_score(y_eval, y_pred_proba)  
print(f"AUC: {auc:.5f}")


AUC: 0.87491


In [15]:
# Alternatively, if you want the importances as a dictionary:
feat_import = model_best.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X_eval.columns, feat_import):
    print(f"{feature}: {importance:.4f}")

Feature Importances:
SegSumT: 0.2582
SegTSeas: 0.0523
SegLowFlow: 0.0549
DSDist: 0.0476
DSMaxSlope: 0.0706
USAvgT: 0.0673
USRainDays: 0.0894
USSlope: 0.0718
USNative: 0.0907
DSDam: 0.0000
Method: 0.1418
LocSed: 0.0553


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

*In terms of AUC values, my model's performance is quite similar to that of Elith et al. My model had an AUC of 0.87491, while Elith et al had an AUC of 0.869. Given the similar AUCs between the two models, it would insinuate that the two models have similar predicitive performance. In addition to predictive performance, you can also compare how computaionally expensive two models are. My model only uses 29 trees, while Elith et al uses 1050 in their model. That means their model takes up considerably more computational power and would be considered more computationally exspensive. Considering the two models deliver a similar predictive performance, my model would win out by being less computationally demanding.*